<a href="https://colab.research.google.com/github/ljkrajewski/HiDream-I1-nf4/blob/main/HiDream_4bq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Run this cell by itself first, then restart the session when requested and run the next cell.

!git clone https://github.com/ljkrajewski/HiDream-I1-nf4.git
%cd HiDream-I1-nf4
!pip install hdi1 --no-build-isolation

In [ ]:
#!python -m hdi1 "A cat holding a sign that says 'hello world'" -m fast
!python -m hdi1.web